In [4]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
import os
import time

### Load all PPP Data that have Lat/Long

In [105]:
df_ppp = pd.read_parquet('ppp_loans_naics_lat_long.parq')

In [106]:
df_ppp.head()

,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,...,OfficialNAICSCode,OfficialNAICSDescription,full_address,ZIP5,postal_code,place_name,state_name,state_code,latitude,longitude
0,2743777209,04/16/2020,0459,PPP,MOBILE PAINT MANUFACTURING COMPANY OF DELAWARE...,"4775 Hamilton Blvd,",Theodore,AL,36582-8509,None,...,325510,Paint and Coating Manufacturing,"4775 Hamilton Blvd, Theodore AL 36582-8509",36582,36582,Theodore,Alabama,AL,30.5444,-88.1807
1,5769087303,04/30/2020,0459,PPP,OAKRIDGE SERVICES LLC,5685B I-10 Industrial Pky \r\nNorth,Theodore,AL,36582,None,...,561730,Landscaping Services,5685B I-10 Industrial Pky \r\nNorth Theodore A...,36582,36582,Theodore,Alabama,AL,30.5444,-88.1807
2,8287317410,05/18/2020,0459,PPP,"BILL BAFF LANDSCAPE, INC.",9830 BELLINGRATH RD,THEODORE,AL,36582,None,...,561730,Landscaping Services,9830 BELLINGRATH RD THEODORE AL 36582,36582,36582,Theodore,Alabama,AL,30.5444,-88.1807
3,3468087300,04/29/2020,0459,PPP,MARK YANCEY,9781 OLD PASCAGOULA RD,THEODORE,AL,36582,None,...,561730,Landscaping Services,9781 OLD PASCAGOULA RD THEODORE AL 36582,36582,36582,Theodore,Alabama,AL,30.5444,-88.1807
4,6138817001,04/06/2020,0459,PPP,"G BRIAN LINDSEY ENTERPRISES, LLC",3820 LE BLANC DR,THEODORE,AL,36582-2536,None,...,561730,Landscaping Services,3820 LE BLANC DR THEODORE AL 36582-2536,36582,36582,Theodore,Alabama,AL,30.5444,-88.1807


In [107]:
len(df_ppp)

5810646

In [108]:
df_ppp.columns

Index(['LoanNumber', 'DateApproved', 'SBAOfficeCode', 'ProcessingMethod',
       'BorrowerName', 'BorrowerAddress', 'BorrowerCity', 'BorrowerState',
       'BorrowerZip', 'LoanStatusDate', 'LoanStatus', 'Term',
       'SBAGuarantyPercentage', 'InitialApprovalAmount',
       'CurrentApprovalAmount', 'UndisbursedAmount', 'FranchiseName',
       'ServicingLenderLocationID', 'ServicingLenderName',
       'ServicingLenderAddress', 'ServicingLenderCity', 'ServicingLenderState',
       'ServicingLenderZip', 'RuralUrbanIndicator', 'HubzoneIndicator',
       'LMIIndicator', 'BusinessAgeDescription', 'ProjectCity',
       'ProjectCountyName', 'ProjectState', 'ProjectZip', 'CD', 'JobsReported',
       'NAICSCode', 'RaceEthnicity', 'UTILITIES_PROCEED', 'PAYROLL_PROCEED',
       'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED', 'REFINANCE_EIDL_PROCEED',
       'HEALTH_CARE_PROCEED', 'DEBT_INTEREST_PROCEED', 'BusinessType',
       'OriginatingLenderLocationID', 'OriginatingLender',
       'OriginatingLen

#### PPP Data - get counts by NAICS codes

In [109]:
df_naics_count = df_ppp[['OfficialNAICSCode', "OfficialNAICSDescription"]].value_counts().reset_index()
df_naics_count.columns = ['OfficialNAICSCode', "OfficialNAICSDescription", 'Count']
print(df_naics_count.head())
print(len(df_naics_count))

  OfficialNAICSCode                           OfficialNAICSDescription   Count
0            722511                          Full-Service Restaurants   220923
1            541110                                 Offices of Lawyers  147005
2            531210          Offices of Real Estate Agents and Brokers  140240
3            621111  Offices of Physicians (except Mental Health Sp...  127969
4            621210                               Offices of Dentists   124796
1057


#### Load the HS6 to NAICS mapping

source: https://www.census.gov/naics/?48967

In [110]:
df_hs_to_naics = pd.read_csv('hs6_to_naics_mapping.csv', dtype='str')
df_hs_to_naics.head()

,Unnamed: 0,hs6,naics
0,0,910211,334518
1,3,160510,311711
2,29,106199,112990
3,39,910212,334518
4,42,910219,334518


In [111]:
df_naics_count_join_hs = df_naics_count.merge(df_hs_to_naics, how='left', left_on='OfficialNAICSCode', right_on='naics')

In [112]:
df_naics_count_join_hs[df_naics_count_join_hs.naics.isna()]

,OfficialNAICSCode,OfficialNAICSDescription,Count,Unnamed: 0,hs6,naics
0,722511,Full-Service Restaurants,220923,NaN,NaN,NaN
1,541110,Offices of Lawyers,147005,NaN,NaN,NaN
2,531210,Offices of Real Estate Agents and Brokers,140240,NaN,NaN,NaN
3,621111,Offices of Physicians (except Mental Health Sp...,127969,NaN,NaN,NaN
4,621210,Offices of Dentists,124796,NaN,NaN,NaN
...,...,...,...,...,...,...
8272,221116,Geothermal Electric Power Generation,35,NaN,NaN,NaN
8290,221113,Nuclear Electric Power Generation,22,NaN,NaN,NaN
8356,921140,"Executive and Legislative Offices, Combined",18,NaN,NaN,NaN
8380,485112,Commuter Rail Systems,13,NaN,NaN,NaN


#### Drop NA rows and extract HS2 code

In [113]:
df_naics_count_join_hs_not_null = df_naics_count_join_hs.dropna(subset=['hs6'])

In [114]:
df_naics_count_join_hs_not_null['hs2'] = [x[0:2] for x in df_naics_count_join_hs_not_null.hs6]

<ipython-input-114-8c5aa2a7ab40>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_naics_count_join_hs_not_null['hs2'] = [x[0:2] for x in df_naics_count_join_hs_not_null.hs6]


In [115]:
df_naics_hs2_counts = df_naics_count_join_hs_not_null.groupby(by='hs2').sum()['Count'].reset_index().sort_values(by='Count', ascending=False)    
print(df_naics_hs2_counts.head())
print(len(df_naics_hs2_counts))

   hs2   Count
20  30  923974
73  84  912676
0   10  565621
2   12  434886
74  85  373229
87


In [120]:
df_naics_count_join_hs_not_null.to_csv('ppp_naics_count_join_hs_not_null.csv')

In [121]:
df_naics_count_join_hs_not_null.naics.value_counts()

325199    391
114111    239
313210    234
331110    190
325180    165
         ... 
212321      1
312113      1
311111      1
111940      1
212312      1
Name: naics, Length: 395, dtype: int64

In [122]:
df_naics_count_join_hs_not_null.columns

Index(['OfficialNAICSCode', 'OfficialNAICSDescription', 'Count', 'Unnamed: 0',
       'hs6', 'naics', 'hs2'],
      dtype='object')

In [127]:
df_naics_count_join_hs_not_null[['hs2', 'naics', 'Count']].drop_duplicates().head()

,hs2,naics,Count
27,10,111150,41943
57,12,111110,20932
78,39,323111,16576
79,48,323111,16576
86,49,323111,16576


In [131]:
df_naics_hs2_counts = df_naics_count_join_hs_not_null[['hs2', 'naics', 'Count']] \
                      .drop_duplicates().groupby('hs2').sum()['Count'].reset_index().sort_values(by='Count', ascending=False)
print(df_naics_hs2_counts.head())
print(len(df_naics_hs2_counts))

   hs2  Count
0   10  80350
2   12  74834
73  84  71873
74  85  62059
79  90  58713
87


In [132]:
df_hs2_chapters = pd.read_csv('hs_code_chapters.csv', dtype='str')
print(len(df_hs2_chapters))

96


In [133]:
df_naics_hs2_counts = df_naics_hs2_counts.merge(df_hs2_chapters, how='left', left_on='hs2', right_on='HS code').sort_values('Count', ascending = False)

In [134]:
df_naics_hs2_counts.columns

Index(['hs2', 'Count', 'HS version', 'Section', 'HS code',
       'HS code description', 'Keep?'],
      dtype='object')

In [135]:
df_naics_hs2_counts.head()

,hs2,Count,HS version,Section,HS code,HS code description,Keep?
0,10,80350,HS17,2,10,CEREALS,Yes
1,12,74834,HS17,2,12,OIL SEEDS AND OLEAGINOUS FRUITS; MISCELLANEOUS...,Yes
2,84,71873,HS17,16,84,"NUCLEAR REACTORS, BOILERS, MACHINERY AND MECHA...",Yes
3,85,62059,HS17,16,85,ELECTRICAL MACHINERY AND EQUIPMENT AND PARTS T...,Yes
4,90,58713,HS17,18,90,"OPTICAL, PHOTOGRAPHIC, CINEMATOGRAPHIC, MEASUR...",Yes


In [136]:
df_naics_hs2_counts[['hs2', 'Count', 'HS code description', 'Keep?']].to_csv('ppp_hs2_chapter_counts.csv', index=False)

#### Check HS2 Chapter Count in 2019 Data

In [17]:
basePath = '/data/common/trade_data/2019_updated/us_customs_2019_cleaned_ignore_multiple_hscode_FULLDESC/parquet_by_month/'
os.listdir(basePath)

['US_Imp_Dec_2019_ignore_multiple_hscode.parq',
 'US_Imp_Aug_2019_ignore_multiple_hscode.parq',
 'US_Imp_Feb_2019_ignore_multiple_hscode.parq',
 'US_Imp_Mar_2019_ignore_multiple_hscode.parq',
 'US_Imp_Jun_2019_ignore_multiple_hscode.parq',
 'US_Imp_Oct_2019_ignore_multiple_hscode.parq',
 'US_Imp_Nov_2019_ignore_multiple_hscode.parq',
 'US_Imp_Jan_2019_ignore_multiple_hscode.parq',
 'US_Imp_Apr_2019_ignore_multiple_hscode.parq',
 'US_Imp_Jul_2019_ignore_multiple_hscode.parq',
 'US_Imp_May_2019_ignore_multiple_hscode.parq',
 'US_Imp_Sep_2019_ignore_multiple_hscode.parq']

In [20]:
#Load Official HSCode List 2019
hs_code_2019 = pd.read_csv('/data/common/trade_data/HS/hs_code_2019_final.csv', dtype='str')
hs_code_2019['HS2'] = [(x[0:2]) for x in hs_code_2019['HS_Code']]
hs_code_2019.head()

,HS_Code,Merged_Description,HS2
0,010121,"Live horses, asses, mules and hinnies ;Horses ...",01
1,010129,"Live horses, asses, mules and hinnies ;Horses ...",01
2,010130,"Live horses, asses, mules and hinnies ;Asses",01
3,010190,"Live horses, asses, mules and hinnies ;Other ;...",01
4,010221,Live bovine animals ;Cattle ;Purebred breeding...,01


In [44]:
df_2019_hs2_counts = pd.DataFrame(data=hs_code_2019.HS2.unique(), columns=['HS2'])
df_2019_hs2_counts.head()

,HS2
0,01
1,02
2,03
3,04
4,05


In [45]:
for f in os.listdir(basePath):
    print(basePath + f)
    colnm = f.split('US_Imp_')[1].split('_2019')[0]
    tempdf = pd.read_parquet(basePath + f)
    tempdf['HS2'] = [x[0:2] for x in tempdf['Cleaned_HS_Code']]
    tempdf2 = tempdf[['HS2', 'System Identity Id']].groupby('HS2').count()['System Identity Id'].reset_index()
    tempdf2.columns = ['HS2', colnm]
    df_2019_hs2_counts = df_2019_hs2_counts = df_2019_hs2_counts.merge(tempdf2, how='left')

/data/common/trade_data/2019_updated/us_customs_2019_cleaned_ignore_multiple_hscode_FULLDESC/parquet_by_month/US_Imp_Dec_2019_ignore_multiple_hscode.parq
/data/common/trade_data/2019_updated/us_customs_2019_cleaned_ignore_multiple_hscode_FULLDESC/parquet_by_month/US_Imp_Aug_2019_ignore_multiple_hscode.parq
/data/common/trade_data/2019_updated/us_customs_2019_cleaned_ignore_multiple_hscode_FULLDESC/parquet_by_month/US_Imp_Feb_2019_ignore_multiple_hscode.parq
/data/common/trade_data/2019_updated/us_customs_2019_cleaned_ignore_multiple_hscode_FULLDESC/parquet_by_month/US_Imp_Mar_2019_ignore_multiple_hscode.parq
/data/common/trade_data/2019_updated/us_customs_2019_cleaned_ignore_multiple_hscode_FULLDESC/parquet_by_month/US_Imp_Jun_2019_ignore_multiple_hscode.parq
/data/common/trade_data/2019_updated/us_customs_2019_cleaned_ignore_multiple_hscode_FULLDESC/parquet_by_month/US_Imp_Oct_2019_ignore_multiple_hscode.parq
/data/common/trade_data/2019_updated/us_customs_2019_cleaned_ignore_multiple

In [46]:
df_2019_hs2_counts

,HS2,Dec,Aug,Feb,Mar,Jun,Oct,Nov,Jan,Apr,Jul,May,Sep
0,01,1117,1337,1200,1144,1223,1237,1138,1532,1208,1620,1377,1313
1,02,4077,4322,3699,4775,4454,3930,3778,4848,5426,4725,4987,3935
2,03,3120,3116,2632,2420,2870,3309,3007,2796,2506,3043,2727,3108
3,04,4137,4104,3806,4228,3580,4338,4261,4693,4249,4374,4668,4738
4,05,741,806,644,685,779,663,693,853,718,821,832,636
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,95,9870,18699,9795,7470,12206,17445,12209,11246,9613,15699,11017,20559
94,96,9359,10749,9254,8158,10793,10447,9642,10375,9446,11657,11197,9988
95,97,2285,2767,2254,2420,2520,2761,2445,2714,2407,2776,2850,2532
96,98,6483,7185,6052,6284,6624,6266,6163,6878,6506,6978,6660,6413


In [52]:
sum_column = df_2019_hs2_counts['Dec'] + df_2019_hs2_counts['Aug'] + df_2019_hs2_counts['Feb'] + df_2019_hs2_counts['Mar'] + df_2019_hs2_counts['Jun'] + df_2019_hs2_counts['Oct'] + \
             df_2019_hs2_counts['Nov'] + df_2019_hs2_counts['Jan'] + df_2019_hs2_counts['Apr'] + df_2019_hs2_counts['Jul'] + df_2019_hs2_counts['May'] + df_2019_hs2_counts['Sep']

In [54]:
df_2019_hs2_counts['Total'] = sum_column

In [56]:
df_2019_hs2_counts.sort_values(by='Total', ascending=False).to_csv('')

,HS2,Dec,Aug,Feb,Mar,Jun,Oct,Nov,Jan,Apr,Jul,May,Sep,Total
82,84,119879,134722,112967,116009,128593,126166,118312,127699,127366,138973,135746,124155,1510587
92,94,67933,64564,55015,48616,60544,61846,62033,68002,57423,67366,66009,62057,741408
83,85,57523,63711,53622,49893,59515,61910,59675,61489,59104,65557,64273,61081,717353
38,39,44261,49177,42048,40168,47957,47010,44158,48939,46669,51581,50735,45808,558511
60,61,25818,38049,29056,27642,30029,38542,28002,34633,28326,38361,31453,35531,385442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,80,397,474,420,401,530,421,401,472,409,481,490,481,5377
97,99,723,438,302,285,529,485,590,135,407,524,475,440,5333
45,46,407,315,395,333,298,302,326,414,324,414,349,311,4188
76,78,284,309,255,241,344,346,319,315,307,385,356,296,3757


In [63]:
df_ppp_counts = pd.read_csv('ppp_hs2_chapter_counts.csv', dtype='str')
df_ppp_counts

,hs2,Count,HS code description,Keep?
0,10,80350,CEREALS,Yes
1,12,74834,OIL SEEDS AND OLEAGINOUS FRUITS; MISCELLANEOUS...,Yes
2,84,71873,"NUCLEAR REACTORS, BOILERS, MACHINERY AND MECHA...",Yes
3,85,62059,ELECTRICAL MACHINERY AND EQUIPMENT AND PARTS T...,Yes
4,90,58713,"OPTICAL, PHOTOGRAPHIC, CINEMATOGRAPHIC, MEASUR...",Yes
...,...,...,...,...
82,11,592,PRODUCTS OF THE MILLING INDUSTRY; MALT; STARCH...,Yes
83,57,433,CARPETS AND OTHER TEXTILE FLOOR COVERINGS,Yes
84,31,225,FERTILISERS,Yes
85,37,175,PHOTOGRAPHIC OR CINEMATOGRAPHIC GOODS,Yes


In [72]:
tempdf = df_2019_hs2_counts[['HS2', 'Total']].merge(df_ppp_counts[['hs2', 'Count', 'HS code description', 'Keep?']], \
                                           how='left', left_on='HS2', right_on='hs2')
tempdf.columns = ['HS2', '2019USImports_HS2_Count', 'HS2', 'PPP_HS2_Count', 'HS2 Chapter Desc', 'Keep?']
tempdf.to_csv('ppp_2019imports_hs2_counts.csv', index=False)

#### Double check PPP NAICS Count join with 2019 Import Concordance from NAICS to HTS10
source :https://www.census.gov/foreign-trade/reference/codes/index.html#concordance

In [50]:
df_hs_to_naics_2019 = pd.read_csv (r'2019_hs_naics_mapping.csv', dtype='str')

In [51]:
df_hs_to_naics_2019['hs6'] = ['0' + x[0:5] if len(x) == 9 else x[0:6] for x in df_hs_to_naics_2019.hts]

In [56]:
df_hs_to_naics_2019 = df_hs_to_naics_2019[['hs6', 'naics']].drop_duplicates()

In [69]:
df_naics_count_join_hs[df_naics_count_join_hs.naics.isna()].merge(df_hs_to_naics_2019, how='left', left_on='naics', right_on='naics')[df_naics_count_join_hs[df_naics_count_join_hs.naics.isna()].merge(df_hs_to_naics_2019, how='left', left_on='naics', right_on='naics').hs6_y.isna()]

,OfficialNAICSCode,OfficialNAICSDescription,Count,Unnamed: 0,hs6_x,naics,hs6_y
0,722511,Full-Service Restaurants,220923,NaN,NaN,NaN,NaN
1,541110,Offices of Lawyers,147005,NaN,NaN,NaN,NaN
2,531210,Offices of Real Estate Agents and Brokers,140240,NaN,NaN,NaN,NaN
3,621111,Offices of Physicians (except Mental Health Sp...,127969,NaN,NaN,NaN,NaN
4,621210,Offices of Dentists,124796,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
657,221116,Geothermal Electric Power Generation,35,NaN,NaN,NaN,NaN
658,221113,Nuclear Electric Power Generation,22,NaN,NaN,NaN,NaN
659,921140,"Executive and Legislative Offices, Combined",18,NaN,NaN,NaN,NaN
660,485112,Commuter Rail Systems,13,NaN,NaN,NaN,NaN
